<a href="https://colab.research.google.com/github/u6yuvi/Courses/blob/main/NLP/UMass-CS685-Advanced-NLP/Week_2_Fixed_Window_NLM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Task:
Given the first three words(window_size) of each sentence, we will try to predict the third word using a fixed window NLM.

Simplification:
1. All text are of equal length.

In [1]:
sentences = ["This a Week-2 Course","Learning Fixed Window NLM","Train a NLM Model","One Last Example Set"]

In [2]:
vocab = {}
inputs = []
for sent in sentences:
  sent_indexes = []
  for word in sent.split(" "):
    if word not in vocab:
        vocab[word] = len(vocab)
    sent_indexes.append(vocab[word])
  inputs.append(sent_indexes)

In [3]:
print(vocab),print(len(vocab))
print(inputs)

{'This': 0, 'a': 1, 'Week-2': 2, 'Course': 3, 'Learning': 4, 'Fixed': 5, 'Window': 6, 'NLM': 7, 'Train': 8, 'Model': 9, 'One': 10, 'Last': 11, 'Example': 12, 'Set': 13}
14
[[0, 1, 2, 3], [4, 5, 6, 7], [8, 1, 7, 9], [10, 11, 12, 13]]


In [4]:
import torch
prefixes = torch.LongTensor([i[:-1] for i in inputs])
labels = torch.LongTensor([i[-1] for i in inputs])
print(prefixes,labels)

tensor([[ 0,  1,  2],
        [ 4,  5,  6],
        [ 8,  1,  7],
        [10, 11, 12]]) tensor([ 3,  7,  9, 13])


In [5]:
import torch.nn as nn

In [6]:
class NLM(nn.Module):
  '''
  Two things to implement
  1. init function to initialize things
  2. Forward Pass
  '''
  def __init__(self,d_embeddings,d_hidden,window_size,len_vocab) -> None:
    super(NLM,self).__init__()

    self.d_embed = d_embeddings 
    self.embeddings = nn.Embedding(len_vocab,d_embeddings)
    # concatenated embeddings > hidden
    self.W_hid = nn.Linear(d_embeddings*window_size,d_hidden)
    # hidden > output probability distribution over vocab
    self.W_out = nn.Linear(d_hidden,len_vocab)

  def forward(self,input): #each input will be a batch of prefixes
    batch_size , window_size = input.size()
    embs = self.embeddings(input) #4x3x5 [batch_size,window_size,d_embeddings]
    #print(embs.shape)

    concat_embs = embs.view(batch_size,window_size*self.d_embed)
    #print(concat_embs.shape)
    hiddens = self.W_hid(concat_embs)

    outs = self.W_out(hiddens)
    #print(outs.shape)

    return outs

network = NLM(d_embeddings=5,d_hidden=10,window_size = 3, len_vocab=len(vocab))
network(prefixes)

num_epochs = 100
learning_rate = 0.01
optimiser  = torch.optim.SGD(params = network.parameters(),lr = learning_rate)
loss_fn = nn.CrossEntropyLoss()

In [7]:
for i in range(num_epochs):
  logits = network(prefixes)
  loss = loss_fn(logits,labels)
  loss.backward()
  optimiser.step()
  optimiser.zero_grad()
  print(f'Loss after Epoch-{i} {loss:.2f}')

Loss after Epoch-0 2.59
Loss after Epoch-1 2.57
Loss after Epoch-2 2.55
Loss after Epoch-3 2.53
Loss after Epoch-4 2.50
Loss after Epoch-5 2.48
Loss after Epoch-6 2.46
Loss after Epoch-7 2.44
Loss after Epoch-8 2.42
Loss after Epoch-9 2.40
Loss after Epoch-10 2.38
Loss after Epoch-11 2.36
Loss after Epoch-12 2.34
Loss after Epoch-13 2.32
Loss after Epoch-14 2.30
Loss after Epoch-15 2.28
Loss after Epoch-16 2.26
Loss after Epoch-17 2.24
Loss after Epoch-18 2.22
Loss after Epoch-19 2.20
Loss after Epoch-20 2.18
Loss after Epoch-21 2.16
Loss after Epoch-22 2.14
Loss after Epoch-23 2.12
Loss after Epoch-24 2.10
Loss after Epoch-25 2.08
Loss after Epoch-26 2.06
Loss after Epoch-27 2.04
Loss after Epoch-28 2.02
Loss after Epoch-29 2.00
Loss after Epoch-30 1.98
Loss after Epoch-31 1.95
Loss after Epoch-32 1.93
Loss after Epoch-33 1.91
Loss after Epoch-34 1.89
Loss after Epoch-35 1.87
Loss after Epoch-36 1.85
Loss after Epoch-37 1.83
Loss after Epoch-38 1.81
Loss after Epoch-39 1.79
Loss after

In [8]:
# our loss has dropped to close to 0.7.
# but is it actually working? let's see

# let's first define a reverse vocabulary mapping (idx>word type)
rev_vocab = dict((idx, word) for (word, idx) in vocab.items())
test_sample = prefixes[0].unsqueeze(0)
logits = network(test_sample)
probs = nn.functional.softmax(logits, dim=1).squeeze()
argmax_idx = torch.argmax(probs).item()
print('This a Week-2 Course", the model predicts "%s" with %0.4f probability' % (rev_vocab[argmax_idx], probs[argmax_idx]))

This a Week-2 Course", the model predicts "Course" with 0.3999 probability
